In [7]:
import requests
import pandas as pd
from datetime import datetime, timedelta
import sys
sys.path.append("../helper_functions")
import duneapi_utils as du
sys.path.pop()

# Define the CSV path and API URL
csv_path = 'outputs/daily_op_circulating_supply.csv'  # Update this path as per your directory structure
api_url = 'https://static.optimism.io/tokenomics/circulatingSupply.txt'

In [8]:
def update_csv_with_api_data(csv_path, api_url):
    # Read the existing CSV file
    try:
        df = pd.read_csv(csv_path)
    except FileNotFoundError:
        print(f"File {csv_path} not found.")
        return

    # Get the current data from the API
    try:
        response = requests.get(api_url)
        response.raise_for_status()
        current_supply = float(response.text.strip())
    except requests.RequestException as e:
        print(f"Error fetching data from API: {e}")
        return

    # Get today's date in the required format
    today = datetime.now().strftime('%Y-%m-%d')

    # Check if today's date is already in the DataFrame
    if today in df['date'].values:
        df.loc[df['date'] == today, 'op_circulating_supply'] = current_supply
    else:
        # Backfill missing dates
        last_date_in_csv = datetime.strptime(df['date'].iloc[-1], '%Y-%m-%d')
        delta = datetime.strptime(today, '%Y-%m-%d') - last_date_in_csv
        for i in range(1, delta.days + 1):
            new_date = last_date_in_csv + timedelta(days=i)
            new_date_str = new_date.strftime('%Y-%m-%d')
            df = df.append({'date': new_date_str, 'op_circulating_supply': current_supply}, ignore_index=True)

    # Save the updated DataFrame back to CSV
    df.to_csv(csv_path, index=False)
    print(f"CSV file updated successfully at {csv_path}")

In [9]:
# Call the function to update the CSV
update_csv_with_api_data(csv_path, api_url)

CSV file updated successfully at outputs/daily_op_circulating_supply.csv


In [10]:
try:
    # Read the CSV file
    df = pd.read_csv(csv_path)

except FileNotFoundError:
    print(f"File {csv_path} not found.")
except Exception as e:
    print(f"An error occurred: {e}")

In [11]:
# Convert to timestamp
df['date'] = pd.to_datetime(df['date'])
df['daily_change_op_supply'] = df['op_circulating_supply'].diff().fillna(0)
# For the first day, the difference is the 'op_circulating_supply' itself since it starts from 0
df['daily_change_op_supply'].iloc[0] = df['op_circulating_supply'].iloc[0]
#print
print(df[df['daily_change_op_supply']>0])
print(df.tail(5))

          date  op_circulating_supply  daily_change_op_supply
0   2022-05-31            214748364.8             214748364.8
267 2023-02-22            317487381.0             102739016.2
346 2023-05-12            335376391.0              20532250.0
365 2023-05-31            644594782.0             309218391.0
414 2023-07-19            679080066.0              34485284.0
426 2023-07-31            716708907.0              37628841.0
457 2023-08-31            798517235.0              81808328.0
490 2023-10-03            880444572.0              81927337.0
527 2023-11-09            880917403.0                472831.0
549 2023-12-01            911294948.0              30377545.0
          date  op_circulating_supply  daily_change_op_supply
566 2023-12-18            911294948.0                     0.0
567 2023-12-19            911294948.0                     0.0
568 2023-12-20            911294948.0                     0.0
569 2023-12-21            911294948.0                     0.0
570 2023

/var/folders/by/kltjc8yd0yz_7_wrtrzhrm9m0000gn/T/ipykernel_59662/1534122912.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['daily_change_op_supply'].iloc[0] = df['op_circulating_supply'].iloc[0]


In [12]:
# Upload to Dune
du.write_dune_api_from_pandas(df, 'daily_op_circulating_supply',\
                             'Daily Snapshots of OP Token Circulating Supply, pulled from: https://static.optimism.io/tokenomics/circulatingSupply.txt')

table at: dune.oplabspbc.dataset_daily_op_circulating_supply
Response status code: 200
Response content: b'{"success":true,"table_name":"daily_op_circulating_supply"}'
